In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from bs4 import BeautifulSoup


In [2]:
questions = pd.read_csv(r'C:\Users\HP\Documents\IITK\DL_Project_Submission\Project 1\stacksample\Questions.csv',encoding='latin-1')


In [5]:
questions.nunique()

Id              1264216
OwnerUserId      630909
CreationDate    1264207
ClosedDate        55949
Score               532
Title           1263995
Body            1264204
dtype: int64

In [6]:
questions.shape

(1264216, 7)

In [7]:
 tags= pd.read_csv(r'C:\Users\HP\Documents\IITK\DL_Project_Submission\Project 1\stacksample\Tags.csv')

In [9]:
tags.nunique()

Id     1264216
Tag      37034
dtype: int64

In [16]:
df_qsn=pd.DataFrame(questions)
df_tags=pd.DataFrame(tags)

In [18]:
merged_df = pd.merge(df_qsn, df_tags, on='Id', how='left')

In [35]:
merged_df.shape

(151196, 8)

In [20]:
merged_df = merged_df.drop_duplicates().reset_index(drop=True)
merged_df = merged_df.dropna().reset_index(drop=True)

In [40]:
merged_df.shape

(151196, 8)

In [37]:
filter_values=merged_df['Tag'].value_counts()[:10].index.tolist()
print(filter_values)

['java', 'php', 'javascript', 'c#', 'python', 'c++', 'html', 'android', 'jquery', 'mysql']


In [38]:
filtered_df = merged_df[merged_df['Tag'].isin(filter_values)]

In [39]:
filtered_df.shape

(46139, 8)

In [41]:
X = filtered_df['Body']
y = filtered_df['Tag']

In [42]:
X.shape,y.shape

((46139,), (46139,))

In [43]:
# Tokenize the questions
tokenizer = Tokenizer(num_words=5000, lower=True)
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)
X = pad_sequences(X, maxlen=100)

In [44]:
#Split the dataset into training and testing sets
y = pd.get_dummies(y).values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [45]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((36911, 100), (9228, 100), (36911, 10), (9228, 10))

In [46]:
# Build and train the LSTM model
model = Sequential()
model.add(Embedding(5000, 128, input_length=X.shape[1]))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))

In [47]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [48]:
model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
1154/1154 [==============================] - 240s 202ms/step - loss: 1.6927 - accuracy: 0.3872
Epoch 2/10
1154/1154 [==============================] - 232s 201ms/step - loss: 1.2381 - accuracy: 0.5626
Epoch 3/10
1154/1154 [==============================] - 232s 201ms/step - loss: 1.0370 - accuracy: 0.6348
Epoch 4/10
1154/1154 [==============================] - 254s 220ms/step - loss: 0.9391 - accuracy: 0.6664
Epoch 5/10
1154/1154 [==============================] - 260s 225ms/step - loss: 0.8746 - accuracy: 0.6853
Epoch 6/10
1154/1154 [==============================] - 258s 224ms/step - loss: 0.8193 - accuracy: 0.6999
Epoch 7/10
1154/1154 [==============================] - 260s 226ms/step - loss: 0.7697 - accuracy: 0.7149
Epoch 8/10
1154/1154 [==============================] - 258s 224ms/step - loss: 0.7267 - accuracy: 0.7235
Epoch 9/10
1154/1154 [==============================] - 244s 212ms/step - loss: 0.6897 - accuracy: 0.7331
Epoch 10/10
1154/1154 [=======================

In [49]:
score, acc = model.evaluate(X_test, y_test, verbose=2)
print('Test accuracy:', acc)

289/289 - 12s - loss: 1.0906 - accuracy: 0.6154 - 12s/epoch - 41ms/step
Test accuracy: 0.6154096126556396


In [50]:
y_pred = (model.predict(X_test) > 0.5).astype(int)

289/289 [==============================] - 9s 31ms/step


In [51]:
from sklearn.metrics import accuracy_score, classification_report

In [52]:
# Print classification report     
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.69      0.75      0.71       640
           1       0.74      0.62      0.68      1013
           2       0.75      0.76      0.75       749
           3       0.43      0.26      0.32       677
           4       0.82      0.66      0.73      1580
           5       0.63      0.36      0.46      1270
           6       0.43      0.26      0.32       618
           7       0.46      0.31      0.37       496
           8       0.67      0.59      0.63      1466
           9       0.83      0.78      0.80       719

   micro avg       0.69      0.55      0.61      9228
   macro avg       0.64      0.53      0.58      9228
weighted avg       0.67      0.55      0.60      9228
 samples avg       0.55      0.55      0.55      9228



C:\Users\HP\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [53]:
tags['Tag'].value_counts()[:10].index.tolist()

['javascript',
 'java',
 'c#',
 'php',
 'android',
 'jquery',
 'python',
 'html',
 'c++',
 'ios']